# HDF5とPyTablesによる巨大配列の操作
　NumPy配列等の配列データを永続化する方法を考える。配列の要素数が小さい場合、np.savetxt,np.save,np.savezなどNumPy組み込みの関数で処理できる。しかし要素数が10億を超える配列は、メモリに読み込むには巨大すぎる。
 
　その代替策がメモリマップだ。配列はファイルに格納され、CPUが必要となった部分が選択的にメモリに読み込まれる(オーバーレイ?)。
  
　ここでは、巨大なデータセットを扱うためのデザインされたPyTablesを使う。これは、[HDF5(Hierarchical Data Format)](https://support.hdfgroup.org/HDF5/)と呼ばれる広く使われているオープンなファイルフォーマットを通して高速なメモリマップを実装している。HDF5ファイルは、POSIX風の階層構造の中に複数のデータセットを格納できる。不必要なシステムメモリの消費を避けながら、データセットのどの部分でも効率的で簡単なアクセスが可能。
 
## 準備
　PyTables3.0以降が必要。Anacondaを使用している場合、
```
conda install tables
```
でPyTablesをインストールできる。

## 手順

### NumPyとPyTablesのインポート

In [2]:
import numpy as np
import tables as tb # tabels for hdf5 file.

### 空のHDF5ファイルを作成

In [4]:
f = tb.open_file('myfile.h5', 'w') # 書き込み権限でmyfile.h5を開く

ValueError: The file 'myfile.h5' is already opened.  Please close it before reopening in write mode.

### 階層のトップレベルグループexperiment1を作成

In [6]:
f.create_group('/', 'experiment1') # ファイル内に"/experiment1"を作成

NodeError: group ``/`` already has a child node named ``experiment1``

### experiment1にメタデータを追加

In [7]:
f.set_node_attr('/experiment1', 'data', '2014-09-01') # data要素としてデータを1つ代入

### グループ内に、1000＊1000の配列array1を格納
　配列(array)を、階層'/experiment1'内に、配列名'array1'として、xの配列を格納する。

In [8]:
x = np.random.rand(1000, 1000) # 1000*1000のスケールで、ランダムデータを入れた、numpy配列を作成
f.create_array('/experiment1', 'array1', x) # 階層名、配列名、格納する配列内容(NumPy)

/experiment1/array1 (Array(1000, 1000)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

### ファイルをクローズして、変更をコミット

In [10]:
f.close()

### 保存HDF5ファイルをオープン

In [12]:
f = tb.open_file('myfile.h5', 'r') # 読み込み権限でh5ファイルを開く

### data要素を取り出してみる

In [14]:
f.get_node_attr('/experiment1', 'data')

'2014-09-01'

### ファイル内パス指定、アクセス
　ルート(/)を起点としてファイル内のパスにアクセスできる。rootから"/"を"."に置き換えてパス指定。

In [15]:
y = f.root.experiment1.array1
# /experiment1/array1
type(y)

tables.array.Array

### 配列の操作性
　NumPyの配列としても操作可能。通常のNumPy配列よりメモリの使用効率は高くなる。

In [16]:
np.array_equal(x[0,:], y[0,:])

True

### 絶対パスの利点
　ノードのパスが実行時に判明するような状況では、絶対パスからノードにアクセスする方法が有効となる。

In [18]:
f.get_node('/experiment1/array1') # 階層/配列名で配列を指定

/experiment1/array1 (Array(1000, 1000)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

### 更新＆ファイル削除

In [19]:
f.close()

In [20]:
import os
os.remove('myfile.h5') # ファイルの削除

## 解説
　複数の配列を一つのファイルに格納できる。巨大な配列を階層構造の中で管理するような大きなプロジェクトでHDF5は活躍する。例えばNASAやNOAAのような科学技術団体などで広く使われている([団体例](www.hdfgroup.org/users.html))。
 
　ノードはグループとデータセットに分かれる。HDF5は内部と外部へのリンクをサポートしている。この昨日は同じ実験やプロジェクトに関するデータを異なるファイルで管理する際に役立つ。
 
　NumPyのスライス文法を使って、さまざまな形式で配列にアクセスできる。